In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [2]:
import matplotlib.pyplot as plt
import seaborn
import pandas as pd

train_data = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
test_data = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")
train_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


## Getting to know the data better

***Checking for only those columns that majorly have null vals in them***

In [3]:
cols_with_null = []
cols = train_data.columns
for i in range(len(cols)):
    if train_data[cols[i]].isnull().sum() / len(train_data[cols[i]]) >=0.5:
        cols_with_null.append((cols[i], train_data[cols[i]].isnull().sum()))


cols_to_drop = [item[0] for item in cols_with_null]
train_data = train_data.drop(columns = cols_to_drop, axis=1)
test_data = test_data.drop(columns = cols_to_drop, axis=1)
train_data

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,...,0,0,0,0,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,Reg,Lvl,AllPub,Inside,...,112,0,0,0,0,4,2010,WD,Normal,142125


***Dropping the cols that contain more than 95% of a certain val***

#Tried to use this for knowing the dist of the data in a col but the data is huge to be plotted so idea dropped
def Freq_plot(df):
    # select_dtypes(include=["object", "categorical", "int64"]) --> returns a dataset with columns that contain these types of data.
    
    data_cols = df.select_dtypes(include=["int64"]).columns  # Creating the array of cols
    data_col_len = len(data_cols)
    print(data_col_len)
    
    # Approach used in here is using pandas and plt
    for i, col in enumerate(data_cols, 1):
        plt.subplot(7, 5, i) # Creating subplots for all cols
  
        # df[column].value_counts() --> counts the frequency of the vals in that column
        
        df[col].value_counts().plot(kind='bar', color='skyblue', edgecolor='black')
        plt.title(f'Distribution of {col}')
        plt.xlabel(col)
        plt.ylabel('Frequency')
        
    plt.show()
        
Freq_plot(train_data)

> Another way of eliminating the cols that contain more than 95% of a certain val in cols with object datatype

In [4]:
obj_cols = train_data.select_dtypes(include = ["object"]).columns
# For dropping the cols that contain redundant data >95%
for col in obj_cols:
    max_val = train_data[col].value_counts(normalize = True).max()
    if max_val>0.9:
        train_data = train_data.drop(columns = col, axis=1)
        test_data = test_data.drop(columns = col, axis=1)
train_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,LotShape,LandContour,LotConfig,Neighborhood,Condition1,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Reg,Lvl,Inside,CollgCr,Norm,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Reg,Lvl,FR2,Veenker,Feedr,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,IR1,Lvl,Inside,CollgCr,Norm,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,IR1,Lvl,Corner,Crawfor,Norm,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,IR1,Lvl,FR2,NoRidge,Norm,...,0,0,0,0,0,12,2008,WD,Normal,250000


In [5]:
test_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,LotShape,LandContour,LotConfig,Neighborhood,Condition1,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Reg,Lvl,Inside,NAmes,Feedr,...,0,0,0,120,0,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,IR1,Lvl,Corner,NAmes,Norm,...,36,0,0,0,0,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,IR1,Lvl,Inside,Gilbert,Norm,...,34,0,0,0,0,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,IR1,Lvl,Inside,Gilbert,Norm,...,36,0,0,0,0,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,IR1,HLS,Inside,StoneBr,Norm,...,82,0,0,144,0,0,1,2010,WD,Normal


In [6]:
train_data.select_dtypes(include=["int64"])

,Id,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,1,60,8450,7,5,2003,2003,706,0,150,...,0,61,0,0,0,0,0,2,2008,208500
1,2,20,9600,6,8,1976,1976,978,0,284,...,298,0,0,0,0,0,0,5,2007,181500
2,3,60,11250,7,5,2001,2002,486,0,434,...,0,42,0,0,0,0,0,9,2008,223500
3,4,70,9550,7,5,1915,1970,216,0,540,...,0,35,272,0,0,0,0,2,2006,140000
4,5,60,14260,8,5,2000,2000,655,0,490,...,192,84,0,0,0,0,0,12,2008,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,7917,6,5,1999,2000,0,0,953,...,0,40,0,0,0,0,0,8,2007,175000
1456,1457,20,13175,6,6,1978,1988,790,163,589,...,349,0,0,0,0,0,0,2,2010,210000
1457,1458,70,9042,7,9,1941,2006,275,0,877,...,0,60,0,0,0,0,2500,5,2010,266500
1458,1459,20,9717,5,6,1950,1996,49,1029,0,...,366,0,112,0,0,0,0,4,2010,142125


**Handling missing vals in the float columns**

In [7]:
from sklearn.impute import SimpleImputer

si = SimpleImputer(strategy = "mean")

imputed_cols_train = si.fit_transform(train_data.select_dtypes(include=["float"]))
imputed_cols_test = si.fit_transform(test_data.select_dtypes(include=["float"]))

train_data[train_data.select_dtypes(include=["float"]).columns] = pd.DataFrame(imputed_cols_train, columns=train_data.select_dtypes(include=["float"]).columns)
test_data[test_data.select_dtypes(include=["float"]).columns] = pd.DataFrame(imputed_cols_test, columns=test_data.select_dtypes(include=["float"]).columns)
test_data.select_dtypes(include=["float"])

,LotFrontage,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,BsmtFullBath,BsmtHalfBath,GarageYrBlt,GarageCars,GarageArea
0,80.0,0.0,468.0,144.0,270.0,882.0,0.0,0.0,1961.000000,1.0,730.0
1,81.0,108.0,923.0,0.0,406.0,1329.0,0.0,0.0,1958.000000,1.0,312.0
2,74.0,0.0,791.0,0.0,137.0,928.0,0.0,0.0,1997.000000,2.0,482.0
3,78.0,20.0,602.0,0.0,324.0,926.0,0.0,0.0,1998.000000,2.0,470.0
4,43.0,0.0,263.0,0.0,1017.0,1280.0,0.0,0.0,1992.000000,2.0,506.0
...,...,...,...,...,...,...,...,...,...,...,...
1454,21.0,0.0,0.0,0.0,546.0,546.0,0.0,0.0,1977.721217,0.0,0.0
1455,21.0,0.0,252.0,0.0,294.0,546.0,0.0,0.0,1970.000000,1.0,286.0
1456,160.0,0.0,1224.0,0.0,0.0,1224.0,1.0,0.0,1960.000000,2.0,576.0
1457,62.0,0.0,337.0,0.0,575.0,912.0,0.0,1.0,1977.721217,0.0,0.0


In [8]:
obj_cols = train_data.select_dtypes(include=["object"]).columns
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for col in obj_cols:
    train_data[col] = le.fit_transform(train_data[col])
    test_data[col] = le.fit_transform(test_data[col])
test_data

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,LotShape,LandContour,LotConfig,Neighborhood,Condition1,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,2,80.0,11622,3,3,4,12,1,...,0,0,0,120,0,0,6,2010,8,4
1,1462,20,3,81.0,14267,0,3,0,12,2,...,36,0,0,0,0,12500,6,2010,8,4
2,1463,60,3,74.0,13830,0,3,4,8,2,...,34,0,0,0,0,0,3,2010,8,4
3,1464,60,3,78.0,9978,0,3,4,8,2,...,36,0,0,0,0,0,6,2010,8,4
4,1465,120,3,43.0,5005,0,1,4,22,2,...,82,0,0,144,0,0,1,2010,8,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,4,21.0,1936,3,3,4,10,2,...,0,0,0,0,0,0,6,2006,8,4
1455,2916,160,4,21.0,1894,3,3,4,10,2,...,24,0,0,0,0,0,4,2006,8,0
1456,2917,20,3,160.0,20000,3,3,4,11,2,...,0,0,0,0,0,0,9,2006,8,0
1457,2918,85,3,62.0,10441,3,3,4,11,2,...,32,0,0,0,0,700,7,2006,8,4


# Now that all columns have been non-nullified we seperate the features and the prediction col

In [9]:
Y = train_data["SalePrice"]
X = train_data.drop(columns = ["Id", "SalePrice"], axis=1)

ids = test_data["Id"]
test_data = test_data.drop(columns = ["Id"], axis=1)

**Splitting the data into training and test split**

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)

In [11]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_Xtrain = scaler.fit_transform(X_train)
scaled_Xtest = scaler.transform(X_test)

scaler_test = StandardScaler()
test_matrix = scaler_test.fit_transform(test_data)

***Creating the model for prediction***

In [12]:
# Trying out ridge regression and lasso regression
from sklearn.linear_model import Ridge, Lasso

ls = Lasso(alpha = 2.84142) # Producing A BETTER MODEL than ridge regression
# rdg = Ridge(alpha = 5)
ls.fit(scaled_Xtrain, Y_train)
pred = ls.predict(test_matrix)
prediction_arr = []
for i in range(len(ids)):
    prediction_arr.append([ids[i], pred[i]])
final_pred = pd.DataFrame(prediction_arr, columns = ["Id", "SalePrice"])
final_pred.to_csv("submission.csv", index=False)
final_pred

,Id,SalePrice
0,1461,109850.241964
1,1462,151308.232855
2,1463,172392.450264
3,1464,192075.317273
4,1465,188616.936715
...,...,...
1454,2915,67454.257640
1455,2916,63804.457911
1456,2917,152233.374973
1457,2918,118757.756521
